In [1]:
import pyomo.environ as pyo
import pandas as pd
import dotenv
import numpy as np
import matplotlib.pyplot as plt
import pyomo.environ as pe
import build_data
from build_data import ext_pyomo_vals, convert_to_dictionary, _auxDictionary
import mysql.connector
import matplotlib.ticker as ticker
import os

In [2]:
conf = dotenv.dotenv_values()
cplex_path = conf["PATH_TO_SOLVER"]

In [3]:
# Set up variables
_import_factor = 400 # -> increase the maximum import capacity
_stor_factor = 600 # -> increase the storage capacity
_specific_date='2019-11-06' # -> Change date
_time_step = 60 # -> Change time step
_update_forecast = False # -> True to have forecast updat in the operation
_save_plots = True # -> True to save the plots

begin_time = 1
end_time = 24
n_scenarios = 3

#Comentarios sobre performance de dias
#Dia 6 corre rapido
#ao fim do dia 13 as previsões sao maiores do q o consumo real
#dia 16/17 é um exemplo de uma pessima previsão

# Method 1 - Forecast

In [4]:
model_forecast = pyo.ConcreteModel("Energy Community")

In [5]:
Data_forecast = build_data.Data() 

In [6]:
model_forecast.t = pe.Set(initialize=np.arange(begin_time, end_time + 1),
                 doc='Time periods')

In [7]:
dict_P_Max_Imp = convert_to_dictionary(Data_forecast.get_data().peers['import_contracted_p_max'][0, begin_time-1:model_forecast.t.last()], t_start=begin_time-1)
dict_P_Max_Exp = convert_to_dictionary(Data_forecast.get_data().peers['export_contracted_p_max'][0, begin_time-1:model_forecast.t.last()], t_start=begin_time-1)

step1_folder = './inputs_database/original_inputs/step1'

Data_forecast.get_data_from_db(specific_date=_specific_date, table="generators_forecast", end = 24*60//_time_step, experiment_id = 17, time_step=_time_step, folder=step1_folder)
Pgen_Forecast_dict = convert_to_dictionary(Data_forecast.get_data().generator['p_forecast'][:, begin_time-1:model_forecast.t.last()], t_start=begin_time-1)

Data_forecast.get_data_from_db(specific_date=_specific_date, table="loads_forecast", end = 24*60//_time_step, experiment_id = 23, time_step=_time_step, folder=step1_folder)
Pload_Forecast_dict = convert_to_dictionary(Data_forecast.get_data().load['p_forecast'][:, begin_time-1:model_forecast.t.last()], t_start=begin_time-1)

dict_C_PImp_Price = convert_to_dictionary(Data_forecast.get_data().peers['buy_price'][0, begin_time-1:model_forecast.t.last()], t_start=begin_time-1)

dict_C_PExp_Price = convert_to_dictionary(Data_forecast.get_data().peers['sell_price'][0, begin_time-1:model_forecast.t.last()], t_start=begin_time-1)

Pgen_Cost_dict = convert_to_dictionary(Data_forecast.get_data().generator['cost_parameter_b'])

Pmax_gen = convert_to_dictionary(Data_forecast.get_data().generator['p_max'])



## Defining Sets:


In [8]:
model_forecast.gens = pe.Set(initialize=np.arange(1, Data_forecast.get_data().generator['p_forecast'].shape[0] + 1),
                    doc='Number of generators')

model_forecast.loads = pe.Set(initialize=np.arange(1, Data_forecast.get_data().load['p_forecast'].shape[0] + 1),
                        doc='Number of loads')

model_forecast.set_scenarios = pyo.Set(initialize=list(range(n_scenarios)), doc='Set of Scenarios')


## Defining Parameters

In [9]:
# Import and Export - Only Depending on time:

model_forecast.Param_P_MaxImp_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_P_Max_Imp, 
                                   within=pyo.NonNegativeReals)

model_forecast.Param_P_MaxExp_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_P_Max_Exp, 
                                   within=pyo.NonNegativeReals)

# Power Load:

model_forecast.Param_Pld_l_t = pyo.Param(model_forecast.loads, model_forecast.t, 
                                   initialize=Pload_Forecast_dict, 
                                   within=pyo.NonNegativeReals)

# Prices - Only Depending on time:

model_forecast.Param_C_ImpPrice_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_C_PImp_Price, 
                                   within=pyo.NonNegativeReals)
model_forecast.Param_c_UpImp_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_C_PImp_Price, 
                                   within=pyo.NonNegativeReals)
model_forecast.Param_c_DownImp_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_C_PImp_Price, 
                                   within=pyo.NonNegativeReals)
model_forecast.Param_C_UpImpPrice_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_C_PImp_Price, 
                                   within=pyo.NonNegativeReals)

model_forecast.Param_C_DownImpPrice_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_C_PImp_Price, 
                                   within=pyo.NonNegativeReals)

model_forecast.Param_C_ExpPrice_t = pyo.Param(model_forecast.t, 
                                   initialize=dict_C_PExp_Price, 
                                   within=pyo.NonNegativeReals)
# Generator Cost:

model_forecast.ParamC_UpGenCost_g_t = pyo.Param(model_forecast.gens, model_forecast.t, 
                                   initialize=Pgen_Cost_dict, 
                                   within=pyo.NonNegativeReals)

model_forecast.ParamC_DownGenCost_g_t = pyo.Param(model_forecast.gens, model_forecast.t, 
                                   initialize=Pgen_Cost_dict, 
                                   within=pyo.NonNegativeReals)

model_forecast.Param_c_Gent_g_t = pyo.Param(model_forecast.gens, model_forecast.t, 
                                   initialize=Pgen_Cost_dict, 
                                   within=pyo.NonNegativeReals)

model_forecast.Param_c_Gen_Up_g_t = pyo.Param(model_forecast.gens, model_forecast.t, 
                                   initialize=Pgen_Cost_dict, 
                                   within=pyo.NonNegativeReals)

model_forecast.Param_c_Gen_Down_g_t = pyo.Param(model_forecast.gens, model_forecast.t, 
                                   initialize=Pgen_Cost_dict, 
                                   within=pyo.NonNegativeReals)
# Generator P Max
model_forecast.Param_P_MaxGen_g = pyo.Param(model_forecast.gens,
                                   initialize=Pmax_gen, 
                                   within=pyo.NonNegativeReals)


## Variables:

In [10]:
# Defining Decision Variables
model_forecast.VarP_OpGen_g_t = pyo.Var(model_forecast.gens, model_forecast.t, domain = pyo.NonNegativeReals, initialize=0)


def UpGen_bounds(model, gen, time):
    lower_bound = 0
    upper_bound = model_forecast.Param_P_MaxGen_g[gen]
    return (lower_bound, upper_bound)

model_forecast.VarR_UpGen_g_t = pyo.Var(model_forecast.gens, model_forecast.t, bounds = UpGen_bounds, domain = pyo.NonNegativeReals, initialize=0)

def DownGen_bounds(model, gen, time):
    lower_bound = 0
    upper_bound = model_forecast.Param_P_MaxGen_g[gen]
    return (lower_bound, upper_bound)

model_forecast.VarR_DownGen_g_t = pyo.Var(model_forecast.gens, model_forecast.t, bounds = DownGen_bounds, domain = pyo.NonNegativeReals, initialize=0)

def R_Up_Import_bounds(model, time):
  lower_bound = 0
  upper_bound = model_forecast.Param_P_MaxImp_t[time]
  return (lower_bound, upper_bound)

model_forecast.VarR_UpImp_t = pyo.Var(model_forecast.t, domain = pyo.NonNegativeReals, bounds = R_Up_Import_bounds, initialize=0)

def R_Down_Import_bounds(model, time):
  lower_bound = 0
  upper_bound = model_forecast.Param_P_MaxImp_t[time]
  return (lower_bound, upper_bound)



model_forecast.VarR_DownImp_t = pyo.Var(model_forecast.t, domain = pyo.NonNegativeReals, bounds = R_Down_Import_bounds, initialize=0)

model_forecast.Var_r_UpGen_g_t_w = pyo.Var(model_forecast.gens, model_forecast.t, model_forecast.set_scenarios, domain = pyo.NonNegativeReals, initialize=0)

model_forecast.Var_r_DownGen_g_t_w = pyo.Var(model_forecast.gens, model_forecast.t, model_forecast.set_scenarios, domain = pyo.NonNegativeReals, initialize=0)

model_forecast.VarP_Op_imp_t = pyo.Var(model_forecast.t, domain = pyo.NonNegativeReals, initialize=0)

model_forecast.Var_r_Up_imp_t_w = pyo.Var(model_forecast.t, model_forecast.set_scenarios, domain = pyo.NonNegativeReals, initialize=0)

model_forecast.Var_r_Down_imp_t_w = pyo.Var(model_forecast.t, model_forecast.set_scenarios, domain = pyo.NonNegativeReals, initialize=0)

model_forecast.VarP_Op_Exp_t = pyo.Var(model_forecast.t, domain = pyo.NonNegativeReals, initialize=0)

model_forecast.VarP_Op_Grid_t = pyo.Var(model_forecast.t, domain = pyo.Reals, initialize=0)

model_forecast.XopGrid_t =  pyo.Var(model_forecast.t, within = pyo.Binary, initialize = 0)



## Constraints:

In [11]:
def rule_power_reserve_Up(model, gen, time, w):
  return model_forecast.Var_r_UpGen_g_t_w[gen, time, w] <= model_forecast.VarR_UpGen_g_t[gen, time]
model_forecast.power_reserve_up_constraint = pyo.Constraint(model_forecast.gens, model_forecast.t, model_forecast.set_scenarios, rule=rule_power_reserve_Up)

def rule_power_reserve_Down(model, gen, time, w):
  return model_forecast.Var_r_DownGen_g_t_w[gen, time, w] <= model_forecast.VarR_DownGen_g_t[gen, time]
model_forecast.power_reserve_Down_constraint = pyo.Constraint(model_forecast.gens, model_forecast.t, model_forecast.set_scenarios, rule=rule_power_reserve_Down)

def rule_power_reserve_Up_import(model,time, w):
  return model_forecast.Var_r_Up_imp_t_w[time, w] <= model_forecast.VarR_UpImp_t[time]
model_forecast.power_reserve_Up_import_constraint = pyo.Constraint(model_forecast.t, model_forecast.set_scenarios, rule=rule_power_reserve_Up_import)

def rule_power_reserve_Down_import(model,time, w):
  return model_forecast.Var_r_Down_imp_t_w[time, w] <= model_forecast.VarR_DownImp_t[time]
model_forecast.power_reserve_Down_import_constraint = pyo.Constraint(model_forecast.t, model_forecast.set_scenarios, rule=rule_power_reserve_Down_import)

def rule_gen_power_max(model, gen, time, w):
  return model_forecast.VarP_OpGen_g_t[gen, time] + model_forecast.Var_r_UpGen_g_t_w[gen, time, w] <= model_forecast.Param_P_MaxGen_g[gen]
model_forecast.gen_power_max_constraint = pyo.Constraint(model_forecast.gens, model_forecast.t, model_forecast.set_scenarios, rule=rule_gen_power_max)

def rule_gen_power_min(model, gen, time, w):
  return model_forecast.VarP_OpGen_g_t[gen, time] - model_forecast.Var_r_DownGen_g_t_w[gen, time, w] >= 0
model_forecast.gen_power_min_constraint = pyo.Constraint(model_forecast.gens, model_forecast.t, model_forecast.set_scenarios, rule=rule_gen_power_min)

def rule_power_imp(model, time, w):
  return model_forecast.VarP_Op_imp_t[time] - model_forecast.Var_r_Up_imp_t_w[time, w] >= 0
model_forecast.power_imp_constraint = pyo.Constraint(model_forecast.t, model_forecast.set_scenarios, rule=rule_power_imp)

def rule_power_reserve_Down_import_Op(model,time, w):
  return model_forecast.VarP_Op_imp_t[time] - model_forecast.Var_r_Down_imp_t_w[time, w] >= 0
model_forecast.power_reserve_Down_import_Op_import_constraint = pyo.Constraint(model_forecast.t, model_forecast.set_scenarios, rule=rule_power_reserve_Down_import_Op)

def rule_power_export(model, time):
  return model_forecast.VarP_Op_Exp_t[time] <= model_forecast.Param_P_MaxExp_t[time]
model_forecast.power_export_constraint = pyo.Constraint(model_forecast.t, rule=rule_power_export)

def rule_imported_power(model, time):
  return model_forecast.VarP_Op_imp_t[time] == (model_forecast.VarP_Op_Grid_t[time] * model_forecast.XopGrid_t[time])
model_forecast.imported_power_constraint = pyo.Constraint(model_forecast.t, rule=rule_imported_power)

def rule_exported_power(model, time):
  return model_forecast.VarP_Op_Exp_t[time] == -1* (model_forecast.VarP_Op_Grid_t[time] * (1-model_forecast.XopGrid_t[time]))
model_forecast.exported_power_constraint = pyo.Constraint(model_forecast.t, rule=rule_exported_power)

def rule_balance_power(model, time, w):
  total_load = sum(model_forecast.Param_Pld_l_t[load, time] for load in model_forecast.loads)
  GenPower = sum(model_forecast.VarP_OpGen_g_t[gen, time] + model_forecast.Var_r_UpGen_g_t_w[gen, time, w] for gen in model_forecast.gens) - model_forecast.Var_r_Down_imp_t_w[time, w]
  return model_forecast.VarP_Op_Grid_t[time] + model_forecast.Var_r_Up_imp_t_w[time, w] - model_forecast.Var_r_Down_imp_t_w[time, w] == total_load - GenPower

model_forecast.balance_power_constraint = pyo.Constraint(model_forecast.t, model_forecast.set_scenarios, rule=rule_balance_power)

## Objective Function

In [12]:

# Objective Function Minimization of Cost
def objective_rule(model):
  CostoRUpGenPower = sum(model_forecast.VarR_UpGen_g_t[gen, time] * model_forecast.ParamC_UpGenCost_g_t[gen, time] for gen in model_forecast.gens for time in model_forecast.t)
  CostoRDownGenPower = sum(model_forecast.VarR_DownGen_g_t[gen, time] * model_forecast.ParamC_DownGenCost_g_t[gen, time] for gen in model_forecast.gens for time in model_forecast.t)
  CostoReservaPowerImport = sum(model_forecast.VarR_UpImp_t[time] * model_forecast.Param_C_UpImpPrice_t[time] + model_forecast.VarR_DownImp_t[time] * model_forecast.Param_C_DownImpPrice_t[time] for time in model_forecast.t)
  FDA = CostoRUpGenPower + CostoRDownGenPower
  CostoGenPower = 0
  CostoPowerImport = 0
  CostoPowerExport = 0
  pi_w = 1/n_scenarios
  for w in model_forecast.set_scenarios:
    CostoGenPower = CostoGenPower + pi_w * sum(model_forecast.Var_r_UpGen_g_t_w[gen, time, w] * model_forecast.Param_c_Gen_Up_g_t[gen, time]  + model_forecast.Var_r_DownGen_g_t_w[gen, time, w] * model_forecast.Param_c_Gen_Down_g_t[gen, time] + model_forecast.VarP_OpGen_g_t[gen, time] * model_forecast.Param_c_Gent_g_t[gen, time] for gen in model_forecast.gens for time in model_forecast.t)
    CostoPowerImport = CostoPowerImport + pi_w *sum(model_forecast.Var_r_Up_imp_t_w[time, w] * model_forecast.Param_c_UpImp_t[time] + model_forecast.Var_r_Down_imp_t_w[time, w] * model_forecast.Param_c_DownImp_t[time] + model_forecast.VarP_Op_imp_t[time] * model_forecast.Param_C_ImpPrice_t[time] for time in model_forecast.t)
    CostoPowerExport = CostoPowerExport + pi_w *sum(model_forecast.VarP_Op_Exp_t[time] * model_forecast.Param_C_ExpPrice_t[time] for time in model_forecast.t)
  FRT = CostoGenPower + CostoPowerImport - CostoPowerExport
  return FDA + FRT

In [13]:

model_forecast.Objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)
solver = pyo.SolverFactory('cplex', executable=cplex_path)
#solver = pyo.SolverFactory('ipopt')
#solver.set_executable(cplex_path, validate=False)
results = solver.solve(model_forecast, tee=True)
print(results)
# Create a list to store the data
data = dict()
columnas = list()

variables = [model_forecast.XopGrid_t]
for variable in variables:
  columnas.append(variable.name)
  data_variable = list()
  for index in variable:
    value = pyo.value(variable[index])
    data_variable.append(value)
  data[variable.name]=data_variable

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data, columns=columnas)
#print(df)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.9.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmpqgu7w5nw.cplex.log' open.
CPLEX> Problem '/tmp/tmp_y0wqsh8.pyomo.lp' read.
Read time = 0.05 sec. (0.17 ticks)
CPLEX> Problem name         : /tmp/tmp_y0wqsh8.pyomo.lp
Objective sense      : Minimize
Variables            :    1368  [Nneg: 1032,  Box: 288,  Free: 24,
                                 Binary: 24]
Objective nonzeros   :    1272
Linear constraints   :    1824  [Less: 1248,  Greater: 504,  Equal: 72]
  Nonzeros           :    4416
  RHS nonzeros       :     456
Quadratic constraints:      48  []
  Linear terms       :      72
  Quadratic terms    :      48
  RHS nonzeros      

# Method 2 - Real

In [14]:
model_real = pyo.ConcreteModel("Energy Community")

In [15]:
Data_real = build_data.Data() 

In [16]:
model_real.t = pe.Set(initialize=np.arange(begin_time, end_time + 1),
                 doc='Time periods')

# Method 3 - Forecast + Real